In [2]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo 

# Integrantes Equipo 8
# Robinson de Leon Morales: 2005055
# Ariadne Selena Romero Rivero: 1957540
# Alberto García García: 2011616
# Samuel Kono Peralta: 1995284

In [3]:
# Load data hojas
# Parametros 
# Demanda
data_m = pd.read_excel("data/Problema2.xlsx", sheet_name="Demanda")

# Productos
data_p = pd.read_excel("data/Problema2.xlsx", sheet_name="Productos")

In [18]:
data_p

,Producto,Costo_produccion,Costo_almacenaje,Inventario_inicial,Inventario_final
0,1,20,0.4,10,50
1,2,25,0.5,0,10
2,3,10,0.3,0,10
3,4,15,0.3,0,10


In [4]:
# Para un ConcreteModel necesitamos los datos al inicio
# Conjuntos

I = data_p["Producto"].unique() # Productos
J = range(1,7) # meses

# Parametros
D = data_m.groupby(by=["Producto"]).sum()
C = data_p.groupby(by=["Producto"]).sum()[["Costo_produccion"]]
A = data_p.groupby(by=["Producto"]).sum()[["Costo_almacenaje"]]
Y = data_p.groupby(by=["Producto"]).sum()[["Inventario_inicial"]]
Z = data_p.groupby(by=["Producto"]).sum()[["Inventario_final"]]

M = 1  #1$ por unidad incrementada
N = 0.5 #$0.5 por unidad decrementadas

In [16]:
# Impresión de parametros :))
Y.loc[2].values[0]

0

# Concrete Model

In [ ]:
# Creamos una variable de la clase ConcreteModel
model = pyo.ConcreteModel()

# Variables de decisión
model.P = pyo.Var(I,J, within=pyo.NonNegativeIntegers, doc="Cantidad de producción del producto I en el mes J")
model.S = pyo.Var(I,J, within=pyo.NonNegativeIntegers, doc="Inventario del producto I al final del mes J")
model.incremento = pyo.Var(J, within=pyo.NonNegativeIntegers, doc="Incremento de la producción entre los meses")
model.decremento = pyo.Var(J, within=pyo.NonNegativeIntegers, doc="Decremento de la producción entre los meses")


# Función objetivo
model.obj = pyo.Objective(
    expr=sum(C.loc[i].values[0] * model.P[i, j] for i in I for j in J) +
         sum(A.loc[i].values[0] * model.S[i, j] for i in I for j in J) +
         sum(M * model.incremento[j] + N * model.decremento[j] for j in J if j > 1),
    sense=pyo.minimize
)


# Restricciones
# Restricción 1: Balance de inventario
model.con1 = pyo.ConstraintList()
for i in I:
    for j in J:
        if j == 1:
            model.con1.add(model.S[i, j] == Y.loc[i].values[0] + model.P[i, j] - D.loc[i].values[j-1])
        else:
            model.con1.add(model.S[i, j] == model.S[i, j-1] + model.P[i, j] - D.loc[i].values[j-1])
                
# -----------------------------------------------------------------------

# Restricción 2: Cumplir el inventario final
# El inventario inicial lo tomamos en cuenta en la restricción 1
model.con2 = pyo.ConstraintList()
for i in I:
    model.con2.add(model.S[i, len(J)] == Z.loc[i].values[0])
    
# ---------------------------------------------------------------------------------------------------------  


# Restricción 3: Restricciones de incremento y decremento
model.con3 = pyo.ConstraintList()
for j in J:
    if j > 1:  # Evita el acceso fuera de rango cuando j=1
        model.con3.add(sum(model.P[i, j] - model.P[i, j - 1] for i in I) 
            == model.incremento[j] - model.decremento[j]
        )
    else:
        model.con3.add(
            sum(model.P[i, j] for i in I) == model.incremento[j]  # No hay decremento en el primer mes
        )
# -------------------------------------------------------------------------------------


KeyError: (1, 1)

In [9]:
model.pprint()

4 Var Declarations
    P : Cantidad de producción del producto I en el mes J
        Size=24, Index={1, 2, 3, 4}*{1, 2, 3, 4, 5, 6}
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (1, 1) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (1, 2) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (1, 3) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (1, 4) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (1, 5) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (1, 6) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (2, 1) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (2, 2) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (2, 3) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (2, 4) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (2, 5) :     0 :  None :  

In [10]:
# Llamamos al solver GLPK y resolvemos el problema
solver = pyo.SolverFactory("glpk")
resultados = solver.solve(model)

In [11]:
resultados

{'Problem': [{'Name': 'unknown', 'Lower bound': 684209.4, 'Upper bound': 684209.4, 'Number of objectives': 1, 'Number of constraints': 34, 'Number of variables': 59, 'Number of nonzeros': 127, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '9', 'Number of created subproblems': '9'}}, 'Error rc': 0, 'Time': 0.06754302978515625}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [12]:
print(f"Valor de la función objetivo: {pyo.value(model.obj)}")

Valor de la función objetivo: 684209.4


In [64]:
print("Producción por mes para cada producto:")
for j in J:
    print(f"Mes {j}:")
    for i in I:
        print(f"  Producto {i}: {pyo.value(model.P[i, j])} unidades")

Producción por mes para cada producto:
Mes 1:
  Producto 1: 1490.0 unidades
  Producto 2: 1300.0 unidades
  Producto 3: 2200.0 unidades
  Producto 4: 2082.0 unidades
Mes 2:
  Producto 1: 3000.0 unidades
  Producto 2: 800.0 unidades
  Producto 3: 1500.0 unidades
  Producto 4: 1772.0 unidades
Mes 3:
  Producto 1: 2000.0 unidades
  Producto 2: 800.0 unidades
  Producto 3: 2900.0 unidades
  Producto 4: 1373.0 unidades
Mes 4:
  Producto 1: 4000.0 unidades
  Producto 2: 1000.0 unidades
  Producto 3: 1800.0 unidades
  Producto 4: 273.0 unidades
Mes 5:
  Producto 1: 2000.0 unidades
  Producto 2: 1100.0 unidades
  Producto 3: 1890.0 unidades
  Producto 4: 1100.0 unidades
Mes 6:
  Producto 1: 2550.0 unidades
  Producto 2: 910.0 unidades
  Producto 3: 1420.0 unidades
  Producto 4: 1210.0 unidades


In [65]:
# Mostrar resultados
for i in I:
    for j in J:
        print(f"Inventario del producto {i} al final del mes {j}: {model.S[i, j].value}")

Inventario del producto 1 al final del mes 1: 0.0
Inventario del producto 1 al final del mes 2: 0.0
Inventario del producto 1 al final del mes 3: 0.0
Inventario del producto 1 al final del mes 4: 0.0
Inventario del producto 1 al final del mes 5: 0.0
Inventario del producto 1 al final del mes 6: 50.0
Inventario del producto 2 al final del mes 1: 0.0
Inventario del producto 2 al final del mes 2: 0.0
Inventario del producto 2 al final del mes 3: 0.0
Inventario del producto 2 al final del mes 4: 0.0
Inventario del producto 2 al final del mes 5: 0.0
Inventario del producto 2 al final del mes 6: 10.0
Inventario del producto 3 al final del mes 1: 0.0
Inventario del producto 3 al final del mes 2: 0.0
Inventario del producto 3 al final del mes 3: 0.0
Inventario del producto 3 al final del mes 4: 0.0
Inventario del producto 3 al final del mes 5: 690.0
Inventario del producto 3 al final del mes 6: 10.0
Inventario del producto 4 al final del mes 1: 682.0
Inventario del producto 4 al final del mes 

In [66]:
print("Total de piezas producidas por mes:")
for j in J:
    total_mes = 0
    for i in I:
        total_mes += pyo.value(model.P[i, j])  # Sumar la producción de cada producto en el mes j
    print(f"Mes {j}: {total_mes} unidades")

Total de piezas producidas por mes:
Mes 1: 7072.0 unidades
Mes 2: 7072.0 unidades
Mes 3: 7073.0 unidades
Mes 4: 7073.0 unidades
Mes 5: 6090.0 unidades
Mes 6: 6090.0 unidades


In [67]:
# Imprimir el incremento y decremento de la producción
print("\nIncremento de la producción por mes:")
for j in J:
    if j > 1:
        print(f"Mes {j}: Incremento = {pyo.value(model.incremento[j])}, Decremento = {pyo.value(model.decremento[j])}")


Incremento de la producción por mes:
Mes 2: Incremento = 0.0, Decremento = 0.0
Mes 3: Incremento = 1.0, Decremento = 0.0
Mes 4: Incremento = 0.0, Decremento = 0.0
Mes 5: Incremento = 0.0, Decremento = 983.0
Mes 6: Incremento = 0.0, Decremento = 0.0
